In [1]:
import numpy as np
import pyvista as pv
import laspy

dcew_laz_fname = '../data/ubrb_lidar/USGS_LPC_ID_FEMAHQ_2018_D18_w1603n2475.laz'

In [2]:
las = laspy.read(dcew_laz_fname)

In [ ]:
points = np.vstack((las.x, las.y, las.z)).transpose()

In [ ]:
point_cloud = pv.PolyData(points)

In [ ]:
# 3. Assign Z values as scalar data for coloring
# The Z coordinate is at index 2 (0=X, 1=Y, 2=Z)
point_cloud['elevation'] = point_cloud.points[:, 2]

# 4. Plot the point cloud, coloring by the 'elevation' scalars
# The 'cmap' argument can be used to choose a specific color map (e.g., 'jet', 'viridis', 'terrain')
# render_points_as_spheres=True makes the points look like small spheres, which can improve visualization quality
point_cloud.plot(
    scalars='elevation',
    cmap='gist_rainbow_r',  # Use a colormap suitable for elevation data
    render_points_as_spheres=True,
    point_size=2,    # Adjust point size as needed
    show_scalar_bar=True,
    cpos='xy'        # Set camera position for a top-down view initially
)

In [4]:
for vlr in las.vlrs:
    print(f"VLR User ID: {vlr.user_id}, Record ID: {vlr.record_id}")

VLR User ID: NIIRS10, Record ID: 4
VLR User ID: NIIRS10, Record ID: 1


In [ ]:
las